# Weather data aggregation for the Finnish residential building stock.

In this example, we'll aggregate ERA5 weather data for Finland
based on national population density statistics for the purposes
of simple lumped-capacitance building thermal modelling of residential
building stock heat demand.
Unlike when calculating PV or wind power production or degree-day heat demand,
we're not interested in the *accumulated* weather, but instead on the *average*
weather over the desired geographical distribution.
For this, we'll exploit the `layout` feature in `atlite`,
except that the "capacity layouts" we'll be using are normalized,
so that they yield weighted average values instead of accumulated ones.

### The weather data parameters we're interested in are:

1. Ambient temperature.
2. Direct solar irradiation for vertical surfaces facing the cardinal directions.
3. Diffuse solar irradiation for vertical surfaces facing the cardinal directions.

This example borrows heavily from the previous examples for
[creating a cutout](https://atlite.readthedocs.io/en/latest/examples/create_cutout.html)
and [historic PV and wind comparison](https://atlite.readthedocs.io/en/latest/examples/historic-comparison-germany.html),
so familiarizing oneself with them might help with following this one.

## Import necessary packages

As usual, we'll need to start by importing the necessary packages.

In [ ]:
import atlite
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)

## Defining the Finland cutout from ERA5

Here, we'll define the `cutout` serving as the basis for our calculations.
For the purposes of this example, we'll be using ERA5 data for Finland for
the year 2011.
We'll use a `GeoDataFrame` for determining the bounding box for Finland.
*(And later for clipping the population density data)*

In [ ]:
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
finland = world.loc[world.iso_a3 == 'FIN']
x1, y1, x2, y2 = finland.bounds.values[0]

In [ ]:
cutout = atlite.Cutout(
    path="finland-2011.nc",
    module="era5",
    x=slice(x1-.2,x2+.2),
    y=slice(y1-.2,y2+.2),
    time="2011"
)

## Preparing the cutout

**This is where the work happens, and preparing the cutout can take long time
depending on your machine and your internet connection when downloading the
required data.**
For comparison, downloading the data for year 2011 while testing took
around 53 minutes.

While techically wind speeds affects building heat demand,
simplified models often replace this interaction with some constant
approximation.
Thus, we're only interested in temperature data for heat losses,
and solar irradiation data for solar heat gains.

In [ ]:
cutout.prepare(features=['influx', 'temperature'])

## Generating layouts

For the population density data, we'll use data from the
[European Environment Agency based on Corine Land Cover 2000](https://www.eea.europa.eu/data-and-maps/data/population-density-disaggregated-with-corine-land-cover-2000-2).
Here's a link to the actual [GeoTiff file](https://www.eea.europa.eu/ds_resolveuid/9TBGV2SUIW).
For reading the GeoTiff into an `xarray.DataArray`, we'll use the
[rioxarray package](https://pypi.org/project/rioxarray/).

In [ ]:
import rioxarray
population_density = rioxarray.open_rasterio("popu01clcv5.tif", masked=True)

Now, the `popu01clcv5.tif` covers the entirety of Europe, which is too large
for our purposes. Thus, we want to clip it to cover only Finland.
However, the population density data uses EPSG:3035 ETRS89/ETRS-LAEA
as its coordinate reference system:

In [ ]:
population_density.rio.crs

whereas the ERA5 data and our Finland `GeoDataFrame` use EPSG:4326

In [ ]:
finland.crs

Thus, we need to do some reprojections.
Since the `popu01clcv5.tif` is quite large, it's actually faster to reproject
the Finland `GeoDataFrame` to EPSG:3035 first, then clip the raster,
and project the clipped raster to EPSG:4326, than to reproject
the entire raster and then clip it.

*(Plotting the raster for Finland takes a few minutes, because the population density data has a ludicurous resolution of 100m x 100m)*

*(The `from_disk=True` preserves memory, as otherwise dealing with the massive raster can result in running out of RAM)*

In [ ]:
# Reproject Finland GeoDataFrame to EPSG:3035
finland_3035 = finland.to_crs("EPSG:3035")

# Clip the raster data and reproject the result back into EPSG:4326,
# also remove some unnecessary dimensions via `squeeze()`
finland_population_density = population_density.rio.clip(
    finland_3035.geometry, from_disk=True
).rio.reproject("EPSG:4326", from_disk=True).squeeze()

# Plot the clipped population density raster.
finland_population_density.plot()
plt.title("Population density, original")

The layout made based on the population density data has an unnecessarily
high resolution for our purposes.
In order to proceed, we need to match the layout to the cutout indices.

In [ ]:
downscaled = finland_population_density.reindex_like(
    cutout.data, method='nearest'
)
downscaled.plot()
plt.title("Population density, downscaled")

Much better.
Finally, since we're interested in the *average* weather instead of *accumulated*
weather, we need to normalize the above raster data to form the final layout.

In [ ]:
layout = downscaled / downscaled.sum()
layout.plot()
plt.title("Population density, downscaled & normalized")

For comparison, we'll also create a uniformly distributed layout.

In [ ]:
uniform = layout.round() + 1.
uniform = uniform / uniform.sum()
uniform.plot()
plt.title("Uniform layout for Finland")

In addition to the population density weighed temperature and the uniformly
distributed temperature, we'll also fetch the temperature for the city
of *Jyväskylä*. 
The heating degree days of Jyväskylä are used by Statistics Finland for
estimating the heating demand of the Finnish residential building stock,
so it acts as a good baseline for out weather data processing.

In [ ]:
import pandas as pd
jyvaskyla = cutout.layout_from_capacity_list(
    pd.DataFrame(data={'x':[25.68], 'y':[62.40], 'Capacity':1.})
)
jyvaskyla.plot()
plt.title("Layout for Jyväskylä")

## Calculating the average weather

Now that we have our layouts ready, we can simply pass them to the `cutout`
to get the weather data we need.
First, let's calculate the ambient temperature and solar irradiation on
vertical surfaces facing different cardinal directions for the population density
weighted layout, the uniform layout, and the city of Jyväskylä.

**Note that we use `layout.fillna(0.)` to fill the missing values with zeroes,**
**and they layouts don't work properly otherwise!**
This wasn't done previously, as examining the raster data is much easier
when the shape of Finland is clearly visible in the plots,
which is not the case after the `NaN` values are replaced with zeroes.

In [ ]:
# Calculate temperature data into a dictionary
temperature_dict = {}
layouts = {'Weighted':layout, 'Uniform':uniform, 'Jyväskylä':jyvaskyla}
for name, lout in layouts.items():
    temperature_dict[name] = cutout.temperature(layout=lout.fillna(0.))\
        .squeeze().to_series()

# Calculate radiation data into dictionaries
irr_total = {}
irr_direct = {}
irr_diffuse = {}
dirs = {'north': 0., 'east': 90., 'south': 180., 'west': 270.}
for name, lout in layouts.items():
    irr_total[name] = {
        d: cutout.irradiation(
            orientation={'slope':90., 'azimuth':az},
            layout=lout.fillna(0.)
        ).squeeze().to_series()
        for d, az in dirs.items()
    }
    irr_direct[name] = {
        d: cutout.irradiation(
            orientation={'slope':90., 'azimuth':az},
            layout=lout.fillna(0.),
            irradiation='direct'
        ).squeeze().to_series()
        for d, az in dirs.items()
    }
    irr_diffuse[name] = {
        d: cutout.irradiation(
            orientation={'slope':90., 'azimuth':az},
            layout=lout.fillna(0.),
            irradiation='diffuse'
        ).squeeze().to_series()
        for d, az in dirs.items()
    }

## Examine the results via plotting.

Let's start with the ambient temperatures, as the plots are simpler.
In the code below, you can use the `fsize` and `time_to_plot` values to
tweak the size and timeframe of the plots.
These settings are used for all the temperature and irradiation plots
under this section.

We recommend plotting results monthly, e.g. `time_to_plot="2011-01"`,
as it works nicely both for temperature and irradiation data.
However, plotting the full year using `time_to_plot="2011"` can help examine
the big picture for temperature and irradiation,
while daily resolution e.g. `time_to_plot="2011-01-01"` can help examine
the solar irradiation details for the different facing walls.
**Note that the data uses UTC+0, while Finland generally uses UTC+2!**

In [ ]:
# Define figure size and timeframe for plotting
fsize = (15,6)
time_to_plot = "2011-01"

# Convert temperature dictionary to dataframe for easier plotting.
temps = pd.DataFrame(temperature_dict)
temps.loc[time_to_plot].plot(
    figsize = fsize,
    title="Ambient temperature comparison",
    ylabel="Ambient temperature [C]",
    xlabel="Datetime"
)

Seems ok. The *Weighted* temperature is noticeably closer to the *Jyväskylä*
temperature than the *Uniform* one, so there's clearly a good reason behind
using the heating degree days from Jyväskylä.
Though, as expected, the *Jyväskylä* temperature is noticeably more volatile
than the averaged temperatures *Weighted* and *Uniform*.

Let's look at the irradiation data next.
First, let's simply examine the total irradiation on the South-facing wall
for the different layouts.

In [ ]:
irrads = pd.DataFrame({
    name: data['south'] for name, data in irr_total.items()
})
irrads.loc[time_to_plot].plot(
    figsize=fsize,
    title="Total irradiation on the south-facing wall",
    xlabel="Datetime",
    ylabel="Total irradiation [W/m2]"
)

Unfortunately, here we can no longer see any meaningful correlation between
the different layouts.
Still, we can examine that the total, direct, and diffuse irradiation all
behave as they should for the *Weighted* layout.

In [ ]:
tot_irrads = pd.DataFrame(irr_total['Weighted'])
tot_irrads.loc[time_to_plot].plot(
    figsize=fsize,
    title="Weighted total irradiation on the walls",
    xlabel="Datetime",
    ylabel="Weighted total irradiation [W/m2]"
)

In [ ]:
dir_irrads = pd.DataFrame(irr_direct['Weighted'])
dir_irrads.loc[time_to_plot].plot(
    figsize=fsize,
    title="Weighted direct irradiation on the walls",
    xlabel="Datetime",
    ylabel="Weighted direct irradiation [W/m2]"
)

In [ ]:
dif_irrads = pd.DataFrame(irr_diffuse['Weighted'])
dif_irrads.loc[time_to_plot].plot(
    figsize=fsize,
    title="Weighted diffuse irradiation on the walls",
    xlabel="Datetime",
    ylabel="Weighted diffuse irradiation [W/m2]"
)

Overall, the irradiation seems to behave as expected.
Direct irradiation peaks at different times for the different walls:
in the morning for the east-facing wall, around noon for the south-facing wall,
in the evening for the west-facing wall, and in both the early morning and
late evening for the north-facing wall in Summer.
Meanwhile, there is no difference in the diffuse irradiation,
as it doesn't matter how the wall is oriented.